## **Run before execution**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import shutil
import pickle
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import keras.backend as K
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Layer, Lambda, Dropout,Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D

##**Load Model**##

In [ ]:
(X_train,Y_train),(X_test,Y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
mnist_model = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/mnist_cnw.model')

### **Generate Probability Vector Predictions**

---



###**Fetching the adversarial images**###

In [ ]:
img = cv2.imread(dir_path+"/"+images[108])
print(images[101])
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2_imshow(img)
img = img/255. - 0.5

# add new axis and expand dimensions to flatten the image.
img = img[...,np.newaxis]
img = np.expand_dims(img,axis=0)

mnist_model.predict(img)

4_1_910.jpg


1/1 [==============================] - 0s 15ms/step


array([[0.03392601, 0.056587  , 0.05392105, 0.15182365, 0.1302794 ,
        0.20763342, 0.11809561, 0.03281816, 0.1538151 , 0.06110056]],
      dtype=float32)

In [ ]:
prediction_vectors = []
dir_path = "/content/drive/MyDrive/Thesis/1000_advs"
images = os.listdir(dir_path)

''' storing the images in x_adv '''
for i in tqdm(range(len(images))):
    img = cv2.imread(dir_path+"/"+images[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255. - 0.5                               # normalization
    img = img[...,np.newaxis]                          # add new axis and expand dimensions to flatten the image.
    img = np.expand_dims(img,axis=0)
    prediction_vectors.append(mnist_model.predict(img))

In [ ]:
pd.DataFrame(prediction_matrix)

,0,1,2,3,4,5,6,7,8,9
0,5.240088e-01,5.239088e-01,2.231372e-02,3.594353e-02,2.955052e-02,8.769359e-02,1.971976e-02,4.563612e-02,5.519088e-02,0.074705
1,8.853688e-06,9.984977e-01,1.305882e-04,1.183733e-03,1.670560e-05,4.007395e-05,9.193034e-06,1.500803e-04,1.100547e-05,0.000052
2,3.427408e-02,5.453588e-01,5.454589e-01,6.816839e-02,4.018464e-02,2.107354e-02,2.144476e-02,5.025938e-02,4.664722e-02,0.035429
3,9.476408e-06,5.605708e-03,2.161767e-04,9.931203e-01,6.183771e-06,6.185657e-04,1.766687e-05,1.400354e-04,1.706434e-04,0.000195
4,1.851252e-02,5.036877e-01,5.067746e-02,4.920112e-02,5.037878e-01,4.590621e-02,4.162702e-02,3.952324e-02,7.256514e-02,0.054861
...,...,...,...,...,...,...,...,...,...,...
9995,4.057567e-02,8.899996e-02,6.567053e-02,1.405336e-01,4.305586e-02,1.902527e-01,3.725423e-02,1.901527e-01,1.486572e-01,0.150044
9996,7.409497e-02,7.783128e-02,1.671375e-01,4.412063e-02,1.048051e-01,9.674668e-02,1.672376e-01,8.244628e-02,1.355636e-01,0.111163
9997,1.450683e-02,4.183247e-02,3.295745e-02,5.268599e-02,3.485709e-02,2.489455e-02,6.965457e-03,4.642922e-01,5.191123e-02,0.464192
9998,2.579621e-02,4.582928e-02,4.683148e-02,5.207474e-02,4.736396e-02,3.181322e-02,1.227444e-02,1.696627e-01,4.627000e-01,0.462600


In [ ]:
# format the shape of matrix to (1000,10)
prediction_vectors = np.array(prediction_vectors)
print("Current shape: " ,prediction_vectors.shape)

prediction_matrix = prediction_vectors[:,0,:]
print("New shape: ", prediction_matrix.shape)

Current shape:  (10000, 1, 10)
New shape:  (10000, 10)


In [ ]:
# save to file 
np.savetxt("10000_vectors.txt",prediction_matrix)

## Phase 1: Distilled Network Training.

---



In [ ]:
# This function creates the on-hot-vector based on the class count and sets the index to 1 as per the given input.
def createOneHotVector(classCount, setIndex):
  vector = np.array(np.zeros(classCount))

  vector[setIndex] = 1

  return vector

In [ ]:
def splitFilePathToGetActualClass(s):

  t = s.split("_")

  return int(t[0])

In [ ]:
# Creat the list of images from the images dir.
image_dist_list = []
one_hot_vectors = []
dir_path = "/content/drive/MyDrive/Thesis/1000_advs"
images = os.listdir(dir_path)

''' storing the images in x_adv '''
for i in tqdm(range(len(images))):
    img = cv2.imread(dir_path+"/"+images[i])
    
    if(img is not None):
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      img = img/255. - 0.5                               # normalization
      image_dist_list.append(img)

      ohv = createOneHotVector(10, splitFilePathToGetActualClass(images[i]))

      print(ohv," : ", splitFilePathToGetActualClass(images[i]))

      one_hot_vectors.append(ohv)

#Shape the images to an array.
image_batch_array = np.array(image_dist_list)
print("X Shape (Input images):", image_batch_array.shape)
print("Y Shape (Actual class):", one_hot_vectors)

In [ ]:
# format the shape of matrix to (1000,10)
onehot_matrix = np.array(one_hot_vectors)
print("Current shape: " ,onehot_matrix.shape)

# save to file 
np.savetxt("/content/drive/MyDrive/Thesis/9996_vectors.txt",onehot_matrix)
print("Saved the matrix successfully")

Current shape:  (9996, 10)
Saved the matrix successfully


In [ ]:
#Load the matrix from the text file.
path_to_txt = "/content/drive/MyDrive/Thesis/9996_vectors.txt"

probability_vector_matrix = np.loadtxt(path_to_txt)

print("Y Shape (Labels):",probability_vector_matrix.shape)

Y Shape (Labels): (9996, 10)


In [ ]:
#Load the model for distillation.
import copy

distilled_model = tf.keras.models.load_model('/content/drive/MyDrive/Thesis/mnist_cnw.model')

In [ ]:
#Freezing: Except for all the dense layers, set the trainable to False.
for layer in distilled_model.layers:
  if("dense" not in layer.name):
    print(layer.name)
    layer.trainable = False

input_3
conv2d_8
conv2d_9
max_pooling2d_4
conv2d_10
conv2d_11
max_pooling2d_5
flatten_2
dropout_2


In [ ]:
#Re-train the model with 10000 images and probability vectors.
distilled_model.fit(x = image_batch_array, y = probability_vector_matrix, epochs=50)

Epoch 1/50
313/313 [==============================] - 2s 6ms/step - loss: 1.7179e-06 - accuracy: 1.0000
Epoch 2/50
313/313 [==============================] - 2s 6ms/step - loss: 1.7680e-06 - accuracy: 1.0000
Epoch 3/50
313/313 [==============================] - 2s 6ms/step - loss: 0.0667 - accuracy: 0.9906
Epoch 4/50
313/313 [==============================] - 2s 6ms/step - loss: 0.0017 - accuracy: 0.9995
Epoch 5/50
313/313 [==============================] - 2s 6ms/step - loss: 0.0019 - accuracy: 0.9993
Epoch 6/50
313/313 [==============================] - 2s 6ms/step - loss: 9.8327e-04 - accuracy: 0.9997
Epoch 7/50
313/313 [==============================] - 2s 6ms/step - loss: 4.9325e-05 - accuracy: 1.0000
Epoch 8/50
313/313 [==============================] - 2s 6ms/step - loss: 2.5657e-05 - accuracy: 1.0000
Epoch 9/50
313/313 [==============================] - 2s 6ms/step - loss: 1.6310e-05 - accuracy: 1.0000
Epoch 10/50
313/313 [==============================] - 2s 6ms/step - loss: 4

In [ ]:
#Save the model.
pickle.dump(distilled_model, open('/content/drive/MyDrive/Thesis/distilled_model_with_one_hot_vector','wb'))

In [ ]:
distilled_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 19, 19, 64)        36928     
                                                           